In [1]:
import sys
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/")
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/Generative_Model")


sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR/recourse/")



In [2]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch.nn as nn
import pickle as pkl
from numpy import linalg as LA


import ML_Models.data_loader as loader
# from utils import _get_input_subset
# from Recourse_Methods.gradient_methods import SCFE
# from utils import get_recourses, get_performance_measures


%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
## Dataset Prep


from torchvision import  datasets, transforms
from torch.utils.data import DataLoader

# data_name = "compas"
data_name = "adult"
# data_name = "german"
n_starting_instances = 1200
compas_dict = {
        "data_path": '../Data_Sets/COMPAS/',
        "filename_train": 'compas-train.csv',
        "filename_test": 'compas-test.csv',
        "label": "risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }

german_dict = {
        "data_path": '../Data_Sets/German_Credit_Data/',
        "filename_train": 'german-train.csv',
        "filename_test": 'german-test.csv',
        "label": "credit-risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
    }

adult_dict = {
        "data_path": "../Data_Sets/Adult/",
        "filename_train": 'adult-train.csv',
        "filename_test": 'adult-test.csv',
        "label": 'income',
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 25,
        "H2": 25,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }


data_meta_dictionaries = {
        "compas": compas_dict, 
        "adult": adult_dict, 
        "german":german_dict
    }
data_meta_info = data_meta_dictionaries[data_name]


dataset_test = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                 filename=data_meta_info["filename_test"],
                                                 label=data_meta_info["label"])
        
dataset_train = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                  filename=data_meta_info["filename_train"],
                                                  label=data_meta_info["label"])


column_names = pd.read_csv(data_meta_info["data_path"] + data_meta_info["filename_train"]).drop(data_meta_info["label"], axis=1).columns



In [4]:
# Data loader

train_loader = DataLoader(dataset_train, batch_size = 32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size = 32, shuffle=False)

data = [i for i in train_loader]
num_input = len(data[0][0][0])


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# do a single pass over the data
def epoch(loader, model, opt=None):
    total_loss, total_err = 0.,0.
    for X,y, ind in loader:
        yp = model(X.view(X.shape[0], -1).to(torch.float32))[:,0]
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

model = nn.Linear(num_input, 1)
opt = optim.SGD(model.parameters(), lr=1e-1)
print("Train Err", "Train Loss", "Test Err", "Test Loss", sep="\t")
for i in range(20):
    train_err, train_loss = epoch(train_loader, model, opt)
    test_err, test_loss = epoch(test_loader, model)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")

    

Train Err	Train Loss	Test Err	Test Loss
0.212925	0.429760	0.209619	0.416164
0.193687	0.401795	0.199558	0.403731
0.185173	0.391062	0.186291	0.393551
0.179230	0.383375	0.181205	0.387143
0.175249	0.377968	0.180431	0.382125
0.175028	0.374016	0.178773	0.378552
0.173563	0.370759	0.176893	0.375831
0.174089	0.368428	0.178662	0.373293
0.172485	0.366408	0.178109	0.371013
0.172154	0.364557	0.178331	0.369768
0.171960	0.363136	0.178994	0.369163
0.171739	0.361900	0.177999	0.367320
0.170937	0.361003	0.179215	0.368446
0.170799	0.360033	0.178883	0.367251
0.169721	0.359289	0.176562	0.364474
0.170025	0.358738	0.175567	0.363303
0.170910	0.357948	0.175124	0.363245
0.169776	0.357450	0.175677	0.362201
0.169721	0.356891	0.174240	0.361668
0.169196	0.356572	0.173908	0.361554


In [6]:
epsilon = 0.2
delta = epsilon * model.weight.detach().sign()

# Testing error on adversarial sample 
def epoch_adv(loader, model, delta):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        X = X.to(torch.float32)
#         tmp = ((2*y.float()-1)[:, None]*delta.squeeze(0)).squeeze(0)
        yp = model(X-((2*y.float()-1)[:, None]*delta.squeeze(0)).squeeze(0)).squeeze(-1)
#         print(yp)
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)
print(epoch_adv(test_loader, model, delta[None,None,:,:]))

(0.9407407407407408, 2.790132334431342)


In [7]:
def epoch_robust(loader, model, epsilon, opt=None):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        yp = model(X.view(X.shape[0], -1).to(torch.float32))[:,0] - epsilon*(2*y.float()-1)*model.weight.norm(1)
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)


model_robust = nn.Linear(num_input, 1)
opt = optim.SGD(model_robust.parameters(), lr=1e-1)
epsilon = 0.2
print("Rob. Train Err", "Rob. Train Loss", "Rob. Test Err", "Rob. Test Loss", sep="\t")
for i in range(20):
    train_err, train_loss = epoch_robust(train_loader, model_robust, epsilon, opt)
    test_err, test_loss = epoch_robust(test_loader, model_robust, epsilon)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")    

Rob. Train Err	Rob. Train Loss	Rob. Test Err	Rob. Test Loss
0.249827	0.560851	0.247872	0.563151
0.247837	0.558472	0.247872	0.557751
0.247837	0.558278	0.247872	0.558047
0.247837	0.558109	0.247872	0.560323
0.247837	0.558191	0.247872	0.557862
0.247837	0.558153	0.247872	0.558181
0.247837	0.558303	0.247872	0.560162
0.247837	0.558216	0.247872	0.558907
0.247837	0.558248	0.247872	0.559119
0.247837	0.558192	0.247872	0.557164
0.247837	0.558168	0.247872	0.559656
0.247837	0.558373	0.247872	0.556316
0.247837	0.558212	0.247872	0.560921
0.247837	0.558270	0.247872	0.560948
0.247837	0.558251	0.247872	0.558048
0.247837	0.558340	0.247872	0.557774
0.247837	0.558226	0.247872	0.558286
0.247837	0.558220	0.247872	0.558997
0.247837	0.558193	0.247872	0.557808
0.247837	0.558211	0.247872	0.564039


In [11]:
# Recourse Method -1 
import torch
import numpy as np
from torch import nn
import datetime


class SCFE:
    
    def __init__(self, classifier, target_threshold: float = 0, _lambda: float = 10.0,
                 lr: float = 0.1, max_iter: int = 500, t_max_min: float = 0.5,
                 step: float = 0.10, norm: int = 1, optimizer: str = 'adam'):
        
        super().__init__()
        self.model_classification = classifier
        self.lr = lr
        self.max_iter = max_iter
        self.optimizer = optimizer
        self.t_max_min = t_max_min
        self.norm = norm
        self.sigmoid = nn.Sigmoid()
        self.target_thres = target_threshold
        self._lambda = _lambda
        self.step = step
    
    def generate_counterfactuals(self, query_instance: torch.tensor, target_class: int = 1) -> torch.tensor:
        """
            query instance: the point to be explained
            target_class: Direction of the desired change. If target_class = 1, we aim to improve the score,
                if target_class = 0, we aim to decrese it (in classification and regression problems).
            _lambda: Lambda parameter (distance regularization) parameter of the problem
        """
        
        if target_class == 1:
            target_prediction = torch.tensor(1).float()
        else:
            target_prediction = torch.tensor(0).float()
        
        output = self._call_model(query_instance.reshape(1, -1))
        output = self._call_model(query_instance)
        
        cf = query_instance.clone().requires_grad_(True)
        
        if self.optimizer == 'adam':
            optim = torch.optim.Adam([cf], self.lr)
        else:
            optim = torch.optim.RMSprop([cf], self.lr)
        
        # Timer
        t0 = datetime.datetime.now()
        t_max = datetime.timedelta(minutes=self.t_max_min)
        
        counterfactuals = []
        while not self._check_cf_valid(output, target_class):
#             print("in...")
            iter = 0
            distances = []
            all_loss = []
            while not self._check_cf_valid(output, target_class) and iter < self.max_iter:
                cf.requires_grad = True
                total_loss, loss_distance = self.compute_loss(self._lambda, cf,
                                                              query_instance,
                                                              target_prediction)
                optim.zero_grad()
                total_loss.backward(retain_graph=True)
                optim.step()
                
                output = self._call_model(cf)
                
                if self._check_cf_valid(output, target_class):
                    counterfactuals.append(cf.detach())
                    distances.append(loss_distance.clone().detach())
                    all_loss.append(total_loss.detach())
                
                iter = iter + 1
            
            output = self._call_model(cf).reshape(1, -1).detach()
            if datetime.datetime.now() - t0 > t_max:
                break

            if self.step == 0.0:  # Don't search over lambdas
                break
            else:
                self._lambda -= self.step

        if not len(counterfactuals):
            print('No CE found')
            cf.detach_()
            return -1
        
        # Choose the nearest counterfactual
        counterfactuals = torch.stack(counterfactuals)
        distances = torch.stack(distances)
        distances = distances.detach()
        index = torch.argmin(distances)
        counterfactuals = counterfactuals.detach()

        ce_star = counterfactuals[index]
        distance_star = distances[index]
        
        
        return ce_star, distance_star
#         return ce_star

    
    def compute_loss(self, _lambda: float, cf_candidate: torch.tensor, original_instance: torch.tensor,
                     target: torch.tensor) -> torch.tensor:
        output = self._call_model(cf_candidate)
        # classification loss
        bce_loss = nn.BCEWithLogitsLoss()
#         print("Testing code : " , output, target)
        loss_classification = bce_loss(output, target)
        # distance loss
        loss_distance = torch.norm((cf_candidate - original_instance), self.norm)
        # full loss
        total_loss = loss_classification + _lambda * loss_distance
        return total_loss, loss_distance

    def _call_model(self, cf_candidate):
        output = self.model_classification(cf_candidate)[0]
#         print(output)
        return output

    def _check_cf_valid(self, output, target_class):
        """ Check if the output constitutes a sufficient CF-example.
            target_class = 1 in general means that we aim to improve the score,
            whereas for target_class = 0 we aim to decrese it.
        """
        if target_class == 1:
            check = output >= self.target_thres
            return check
        else:
            check = output <= self.target_thres
            return check
        


In [12]:
# Recourse Method 2
import Recourse_Methods.Generative_Model.model as model_vae
from numpy import linalg as LA

# Second class of counter-factual explanation methods         
class CCHVAE:

    def __init__(self, classifier, model_vae, target_threshold: float = 0,
                 n_search_samples: int = 10000, p_norm: int = 1,
                 step: float = 0.05, max_iter: int = 1000, clamp: bool = True):
        
        super().__init__()
        self.classifier = classifier
        self.generative_model = model_vae
        self.n_search_samples = n_search_samples
        self.p_norm = p_norm
        self.step = step
        self.max_iter = max_iter
        self.clamp = clamp
        self.target_treshold = target_threshold

    def hyper_sphere_coordindates(self, instance, high, low):
    
        """
        :param n_search_samples: int > 0
        :param instance: numpy input point array
        :param high: float>= 0, h>l; upper bound
        :param low: float>= 0, l<h; lower bound
        :param p: float>= 1; norm
        :return: candidate counterfactuals & distances
        """
    
        delta_instance = np.random.randn(self.n_search_samples, instance.shape[1])
        dist = np.random.rand(self.n_search_samples) * (high - low) + low  # length range [l, h)
        norm_p = LA.norm(delta_instance, ord=self.p_norm, axis=1)
        d_norm = np.divide(dist, norm_p).reshape(-1, 1)  # rescale/normalize factor
        delta_instance = np.multiply(delta_instance, d_norm)
        candidate_counterfactuals = instance + delta_instance
    
        return candidate_counterfactuals, dist

    def generate_counterfactuals(self, query_instance: torch.tensor, target_class: int = 1) -> torch.tensor:
        """
        :param instance: np array
        :return: best CE
        """  #

        # init step size for growing the sphere
        low = 0
        high = low + self.step

        # counter
        count = 0
        counter_step = 1
        query_instance = query_instance.detach().numpy()

        # get predicted label of instance
        self.classifier.eval()
        instance_label = 1 - target_class
        # vectorize z
        z = self.generative_model.encode_csearch(torch.from_numpy(query_instance).float()).detach().numpy()
        z_rep = np.repeat(z.reshape(1, -1), self.n_search_samples, axis=0)

        while True:
            count = count + counter_step
            if count > self.max_iter:
                candidate_counterfactual_star = np.empty(query_instance.shape[0], )
                candidate_counterfactual_star[:] = np.nan
                distance_star = -1
                print('No CE found')
                break

            # STEP 1 -- SAMPLE POINTS on hypersphere around instance
            latent_neighbourhood, _ = CCHVAE.hyper_sphere_coordindates(self, z_rep, high, low)
            
#             print("-->> ", latent_neighbourhood)
            x_ce = self.generative_model.decode_csearch(torch.from_numpy(latent_neighbourhood).float()).detach().numpy()
            
            
            # why do we have this?
#             print(x_ce)
            if self.clamp:
                x_ce = x_ce.clip(-4, 4)

            # STEP 2 -- COMPUTE l1 & l2 norms
            if self.p_norm == 1:
                distances = np.abs((x_ce - query_instance)).sum(axis=1)
            elif self.p_norm == 2:
                distances = LA.norm(x_ce - query_instance, axis=1)
            else:
                print('Distance not defined yet')
            
            # counterfactual labels
#             print(self.classifier(torch.from_numpy(x_ce).float()).detach().numpy())
            y_candidate = torch.stack([torch.tensor([int(i[0])]) for i in self.classifier(torch.from_numpy(x_ce).float()).detach().numpy() > 0])
            #print("Y_Cands : ", self.classifier(torch.from_numpy(x_ce).float()).detach().numpy())
            indeces = np.where(y_candidate != instance_label)[0]
#             print("Indeces : ", indeces)
            candidate_counterfactuals = x_ce[indeces]
            candidate_dist = distances[indeces]
            
            if len(candidate_dist) == 0:  # no candidate found & push search range outside
                low = high
                high = low + self.step
            elif len(candidate_dist) > 0:  # certain candidates generated
                min_index = np.argmin(candidate_dist)
                candidate_counterfactual_star = candidate_counterfactuals[min_index]
                distance_star = np.abs(candidate_counterfactual_star - query_instance).sum()
                break

        return torch.tensor(candidate_counterfactual_star), torch.tensor(distance_star)
#         return  torch.tensor(distance_star)     
    
    
    
    
    

In [13]:
def _get_input_subset(model_sub, inputs: torch.tensor,
                      subset_size: int = 100,
                      decision_threshold: float = 0) -> torch.tensor:
    
    """
    Get negatively classified inputs & return their predictions
    """
    
    yhat = (model_sub(inputs) > decision_threshold) * 1
    check = (model_sub(inputs) < decision_threshold).detach().numpy()
    selected_indices = np.where(check)[0]
    input_subset = inputs[selected_indices]
    predicted_label_subset = yhat[selected_indices]
    return input_subset, predicted_label_subset #[0:subset_size]

In [17]:
# scfe = SCFE(classifier=model, lr=1e-2, _lambda=0.10, step=0.00, max_iter=40000, target_threshold=0)
# scfe_r = SCFE(classifier=model_robust, lr=1e-2, _lambda=0.10, step=0.00, max_iter=40000, target_threshold=0)
# f(x) = sign(wTX + b)
scfe = SCFE(classifier=model, lr=1e-3, _lambda=0.00, step=0.00, max_iter=10000, target_threshold=0)
scfe_r = SCFE(classifier=model_robust, lr=1e-3, _lambda=0.00, step=0.00, max_iter=10000, target_threshold=0)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
subset_size = 200
testloader = DataLoader(dataset_test,
                        batch_size=data_meta_info["n_starting_instances"],
                        shuffle=False)

data_iter = iter(testloader)
inputs, labels, indeces = data_iter.next()
inputs = inputs.to(device).float()
inputs, predicted_classes = _get_input_subset(model, inputs, subset_size, decision_threshold=0)

cfe_scfe = []
for i in range(len(inputs)):
    if i%100 == 0:
        print("Gotten {} done...".format(i))
    cfe, ctr_val = scfe.generate_counterfactuals(
                    query_instance=inputs[i],
                    target_class=1)
    if ctr_val.item() > 0:
        cfe_scfe.append(cfe)
        
# distance_scfe = [scfe.generate_counterfactuals(
#                     query_instance=inputs[i],
#                     target_class=1).item() for i in range(len(inputs)) if scfe.generate_counterfactuals(
#                     query_instance=inputs[i],
#                     target_class=1) > 0]



Gotten 0 done...
Gotten 100 done...
Gotten 200 done...
Gotten 300 done...
Gotten 400 done...
Gotten 500 done...
Gotten 600 done...
Gotten 700 done...
Gotten 800 done...
Gotten 900 done...


In [19]:
# Load the VAEs

vae_path = "../Recourse_Methods/Generative_Model/Saved_Models/"

input_size = dataset_train.get_number_of_features()

vae_model = model_vae.VAE_model(input_size,
                                data_meta_info['activFun'],
                                data_meta_info['d'],
                                data_meta_info['H1'],
                                data_meta_info['H2'])

data_meta_info["vae_path"] = vae_path + f"vae_{data_name}.pt"
# print(vae_path + f"vae_{data_name}.pt")

vae_model.load_state_dict(torch.load(data_meta_info["vae_path"]))
        
testloader = DataLoader(dataset_test,
                        batch_size=data_meta_info["n_starting_instances"],
                        shuffle=True)
        
trainloader = DataLoader(dataset_train,
                         batch_size=data_meta_info["n_starting_instances"],
                         shuffle=True)


# This method makes the "Manifold assumption" and uses random search in latent space
cchvae = CCHVAE(classifier=model, model_vae=vae_model, step=4.5, max_iter=10000, target_threshold=0)
cchvae_r = CCHVAE(classifier=model_robust, model_vae=vae_model, step=4.5, max_iter=10000, target_threshold=0)







In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
subset_size = 1
testloader = DataLoader(dataset_test,
                        batch_size=data_meta_info["n_starting_instances"],
                        shuffle=False)

data_iter = iter(testloader)
inputs, labels, indeces = data_iter.next()
inputs = inputs.to(device).float()
inputs, predicted_classes = _get_input_subset(model_robust, inputs, subset_size, decision_threshold=0)
print(inputs.shape)
print(inputs[0].reshape(1, -1))

cfe_cchvae = []
for i in range(len(inputs)):
    if i%10 == 0:
        print("Gotten {} done...".format(i))
    cfe, ctr_val = cchvae.generate_counterfactuals(
                    query_instance= inputs[i].reshape(1, -1),
                    target_class= 1 )
    if ctr_val.item() > 0:
        cfe_cchvae.append(cfe)
        
        
        

torch.Size([1200, 13])
tensor([[0.1370, 0.1511, 0.5333, 0.0000, 0.0000, 0.1939, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 1.0000]])
Gotten 0 done...
Gotten 10 done...
Gotten 20 done...
Gotten 30 done...
Gotten 40 done...
Gotten 50 done...
Gotten 60 done...
Gotten 70 done...
Gotten 80 done...
Gotten 90 done...
Gotten 100 done...
Gotten 110 done...
Gotten 120 done...
Gotten 130 done...
Gotten 140 done...
Gotten 150 done...
Gotten 160 done...
Gotten 170 done...
Gotten 180 done...
Gotten 190 done...
Gotten 200 done...
Gotten 210 done...
Gotten 220 done...
Gotten 230 done...
Gotten 240 done...
Gotten 250 done...
Gotten 260 done...
Gotten 270 done...
Gotten 280 done...
Gotten 290 done...
Gotten 300 done...
Gotten 310 done...
Gotten 320 done...
Gotten 330 done...
Gotten 340 done...
Gotten 350 done...
Gotten 360 done...
Gotten 370 done...
Gotten 380 done...
Gotten 390 done...
Gotten 400 done...
Gotten 410 done...
Gotten 420 done...
Gotten 430 done...
Gotten 440 done...
Gotten 45

In [167]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
subset_size = 5
testloader = DataLoader(dataset_test,
                        batch_size=data_meta_info["n_starting_instances"],
                        shuffle=False)

data_iter = iter(testloader)
inputs, labels, indeces = data_iter.next()
inputs = inputs.to(device).float()
inputs, predicted_classes = _get_input_subset(model_robust, inputs, subset_size, decision_threshold=0)


cfe_cchvae = []
for i in range(len(inputs)):
    if i%100 == 0:
        print("Gotten {} done...".format(i))
    ctr_val = scfe_r.generate_counterfactuals(
                    query_instance=inputs[i],
                    target_class=1)
    if ctr_val > 0:
        distance_scfe_r.append(ctr_val)
        
# distance_scfe_r = [scfe_r.generate_counterfactuals(
#                     query_instance=inputs[i],
#                     target_class=1).item() for i in range(len(inputs)) if scfe_r.generate_counterfactuals(
#                     query_instance=inputs[i],
#                     target_class=1) > 0]


Gotten 0 done...
Gotten 100 done...
Gotten 200 done...
Gotten 300 done...
Gotten 400 done...
Gotten 500 done...
Gotten 600 done...
Gotten 700 done...
Gotten 800 done...
Gotten 900 done...
Gotten 1000 done...
Gotten 1100 done...


In [168]:
len(distance_scfe_r)

1200

In [177]:
model_name = ["base"]*len(distance_scfe) + ["robust"]*len(distance_scfe_r) # + len(distance_scfe))
cost_vals = [i.item() for i in distance_scfe] + [i.item() for i in distance_scfe_r]


In [180]:
model_name

['base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 'base',
 

In [182]:
scfe_data_df = pd.DataFrame(list(zip(cost_vals, model_name)), columns=["Cost", "Model"])



In [187]:
scfe_data_df.to_csv("{}_scfe_data_df.csv".format(data_name))

In [113]:
sum(distance_scfe)/len(distance_scfe)

ZeroDivisionError: division by zero

In [185]:
# Validity : 

# Step 1 : Get counter-factuals for baseline
# Step 2 : Apply to robust models to see if the prediction is 1
